In [1]:
from pyspark.sql import SparkSession
import glob
import numpy as np
import pandas as pd
import os
import concurrent.futures
from tqdm import tqdm

In [2]:
#Set Environment accordingly
os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home/"
os.environ['SPARK_HOME'] = "/Users/simran/Downloads/spark-3.5.5-bin-hadoop3/"

In [3]:
directory_path = "./data/full_history"
file_pattern = "*.csv"

In [ ]:

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("BigDataProject1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "15g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "20g") \
    .config("spark.default.parallelism", "100") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.serializer", "org.apache.spark.serializer.JavaSerializer") \
    .getOrCreate()


In [14]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

schema = StructType([
    StructField("date", DateType(), True),
    StructField("volume", DoubleType(), True),  
    StructField("open", DoubleType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("close", DoubleType(), True),
    StructField("adj close", DoubleType(), True),
    StructField("StockName", StringType(), True)
])

In [15]:
# Load datasets as DataFrames
history_df = spark.read.format("csv") \
     .option("header", "true") \
     .option("treatEmptyValuesAsNulls", "true") \
     .option("schema",schema) \
     .load(f"{directory_path}/{file_pattern}")


history_df.show(5)

+----------+-------+-----------------+------------------+------------------+------------------+------------------+---------+
|      date| volume|             open|              high|               low|             close|         adj close|StockName|
+----------+-------+-----------------+------------------+------------------+------------------+------------------+---------+
|2023-12-28|5703600|30.14999961853028|30.239999771118164|29.950000762939453| 30.18000030517578| 30.18000030517578|      HPQ|
|2023-12-27|5127900|30.39999961853028|30.520000457763672|30.209999084472656|30.239999771118164|30.239999771118164|      HPQ|
|2023-12-26|4853600|30.13999938964844|30.549999237060547|30.059999465942383| 30.40999984741211| 30.40999984741211|      HPQ|
|2023-12-22|4476400|30.21999931335449|30.479999542236328| 30.06999969482422| 30.18000030517578| 30.18000030517578|      HPQ|
|2023-12-21|6529200|30.07999992370605| 30.21999931335449| 29.90999984741211| 30.13999938964844| 30.13999938964844|      HPQ|


In [17]:
# Remove duplicates
history_df = history_df.dropDuplicates()

# Handle missing values for both open and close simultaneously
history_df = history_df.na.fill({
    "open": np.nan,
    "close": np.nan
})
history_df = history_df.replace(float('nan'), None)

history_df.cache()


DataFrame[date: string, volume: string, open: string, high: string, low: string, close: string, adj close: string, StockName: string]

In [8]:
history_df.describe().show()

25/04/26 15:15:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
history_df.count()


29677722

In [ ]:
#Convert Data to Pandas DataFrame to plot visualisations

all_files = glob.glob("./data/full_history/*.csv")

history_df_pd = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)



In [20]:
#Data Visualisations

#Time series visualisation given stock name

import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.dates as mdate
from IPython.display import display


# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=sorted(list(history_df_pd['StockName'].unique())),
    description='StockNames:'
)

# Function to update the plot based on dropdown selection
def update_plot(category):
    plt.clf() # Clear previous plot
    filtered_df = history_df_pd[history_df_pd['StockName'] == category]
    filtered_df.loc[:,'date'] = pd.to_datetime(filtered_df['date'])
    # Plotting closing prices for a single stock
    sorted_filtered_df = filtered_df.sort_values(by='date')
    plt.plot(sorted_filtered_df['date'], sorted_filtered_df['close'])
    plt.title(f'Plot for {category}')
    plt.xlabel('Time')
    plt.ylabel('Price')
    locator = mdate.YearLocator()
    plt.gca().xaxis.set_major_locator(locator)

    plt.gcf().autofmt_xdate()


# Observe the dropdown value and update the plot
out = widgets.interactive_output(update_plot, {'category': dropdown})

# Display the dropdown and the initial plot

display(dropdown, out)

Dropdown(description='StockNames:', options=('RIV', 'ANTE', 'CSCO', 'PRI', 'NZF', 'HLNE', 'HUBS', 'GPL', 'PSM'…

Output()

In [21]:
#Data Visualisations

#Moving Averages visualisation given stock name


# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=sorted(list(history_df_pd['StockName'].unique())),
    description='StockNames:'
)

# Function to update the plot based on dropdown selection
def update_plot(category):
    plt.clf() # Clear previous plot
    filtered_df = history_df_pd[history_df_pd['StockName'] == category]
    filtered_df.loc[:,'date'] = pd.to_datetime(filtered_df['date'])
    # Plotting closing prices for a single stock
    sorted_filtered_df = filtered_df.sort_values(by='date')
    ma100 = sorted_filtered_df.close.rolling(100).mean()
    plt.plot(sorted_filtered_df['date'], sorted_filtered_df['close'])
    plt.plot(sorted_filtered_df['date'],ma100, 'r')
    plt.title(f'Plot for {category}')
    plt.xlabel('Time')
    plt.ylabel('Price')
    locator = mdate.YearLocator()
    plt.gca().xaxis.set_major_locator(locator)

    plt.gcf().autofmt_xdate()


# Observe the dropdown value and update the plot
out = widgets.interactive_output(update_plot, {'category': dropdown})

# Display the dropdown and the initial plot

display(dropdown, out)

Dropdown(description='StockNames:', options=('RIV', 'ANTE', 'CSCO', 'PRI', 'NZF', 'HLNE', 'HUBS', 'GPL', 'PSM'…

Output()

In [ ]:
# Correlation Heatmap
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation matrix
corr_matrix = history_df_pd[['open', 'high', 'low', 'close', 'adj close', 'volume']].corr()

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap of Stock Features")
plt.show()


In [ ]:
#Daily Returns Distribution

stock = 'AAPL'  # Replace with desired stock
df = history_df_pd[history_df_pd['StockName'] == stock].copy()
df = df.sort_values(by='date')
df['daily_return'] = df['close'].pct_change()

plt.figure(figsize=(10, 6))
sns.histplot(df['daily_return'].dropna(), bins=100, kde=True, color='blue')
plt.title(f"Daily Return Distribution for {stock}")
plt.xlabel("Daily Return")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


In [ ]:
#Boxplots for Outlier Detection
plt.figure(figsize=(12, 6))
sns.boxplot(data=history_df_pd[['open', 'high', 'low', 'close', 'adj close', 'volume']])
plt.title("Boxplot of Stock Features")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
#Top 10 Most Volatile Stocks in the Model
df = history_df_pd.copy()
df['daily_return'] = df.groupby('StockName')['close'].pct_change()
volatility = df.groupby('StockName')['daily_return'].std().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 5))
volatility.plot(kind='bar', color='crimson')
plt.title("Top 10 Most Volatile Stocks (Std Dev of Daily Returns)")
plt.ylabel("Volatility")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()
